# Lab 3 — clustering

In [6]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import json
import matplotlib.pyplot as plt
import numpy as np
import pickle

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool

In [2]:
output_notebook()

%matplotlib inline
plt.style.use("ggplot")

Loading BokehJS ...

## Exercise 3.11

In [3]:
with open("tags_coordinates.pickle", "rb") as f:
    tags_coordinates = pickle.load(f, encoding="utf-8")

tags = list(tags_coordinates.keys())    
X = np.asarray(list(tags_coordinates.values()))

In [19]:
kmeans = [KMeans(n_clusters=i, random_state=0).fit(X) for i in range(2, 6)]

In [39]:
pca = PCA(n_components=2)
pca.fit(X)

t = pca.transform(X)

In [49]:
def get_color(x):
    if x == 0:
        return "#0000FF"
    elif x == 1:
        return "#00FF00"
    elif x == 2:
        return "#FF0000"
    elif x == 3:
        return "#FFFF00"
    else:
        return "#00FFFF"

In [50]:
def plot(i):
    source = ColumnDataSource(
        data={
            "x": t[:, 0],
            "y": t[:, 1],
            "name": tags,
            "color": [get_color(x) for x in kmeans[i].labels_],
        })
    hover = HoverTool(
        tooltips=[
            ("Name", "@name"),
        ])
    tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

    p = figure(plot_width=960, tools=tools)
    p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
    show(p, notebook_handle=True)

In [51]:
plot(0)

In [52]:
plot(1)

In [53]:
plot(2)

In [54]:
plot(3)